https://en.wikipedia.org/wiki/General_linear_model

# Least Squares Method:
## Surface Polynomial Regression
Lets expand previous concepts, exapnding our previos function to multiple dimmensions: <br>
${\displaystyle f(x,{\boldsymbol {\beta }})=\sum _{j=1}^{m}\beta _{j}\phi _{j}(x)} \quad\rightarrow\quad \mathbb{R}^{n}\quad\rightarrow\quad
y = {\displaystyle f(\vec{x},\vec{{\boldsymbol {\beta }}})=\sum _{j=1}^{m}\vec {\beta} _{j}\phi _{j}(\vec{x}),}$ <br><br>

Note that function below is an scalar function with vectorial codomain: $\quad y = f(\vec{x},\vec{{\boldsymbol {\beta }}}): \quad \mathbb{R}^{n} \rightarrow \mathbb{R} $

$ {\begin{bmatrix}
  {y}_{1}
 \cr {y}_{2}
 \cr  {y}_{3}
 \cr  ...
 \cr  {y}_{n}\end{bmatrix}}$
 $=$
${\begin{bmatrix}1&\vec {x}_{1}&\vec {x}_{1}^{2}&... &\vec {x}_{1}^{r}\cr1&\vec {x}_{2}&\vec {x}_{2}^{2}&... &\vec {x}_{2}^{r}\cr1&\vec {x}_{3}&\vec {x}_{3}^{2}&... &\vec {x}_{3}^{r}\cr...\cr1&\vec {x}_{n}&\vec {x}_{n}^{2}&\dots &\vec {x}_{n}^{r}\end{bmatrix}}$
$+$
${\begin{bmatrix}\vec {\beta} _{0}\\\vec {\beta} _{1}\\\vec {\beta} _{2}\\\vdots \\\vec {\beta} _{r}\end{bmatrix}}$


here we are trying to expand the Polynomial regression, but polynomial function has all combinations of independent coordinates.
Calculating powers as scalar products we get something like:
<br>x^2 = x^2 + 2xy + y^2
<br>x^3 = x^3 + 3x^2y + 3xy^2 +y^3<br>
This could causes some disturbance when we get beta coeff

<br><br><br>
WAHT IF WE DEFINE THE POWER AS A NEW DIMENSION?<br>
-THER WE WOULD HAVE FORE EXAMPLE<br>
Z = Z(X,Y,g) where g is the max_grade of the expoenet<br>

$\begin{bmatrix}z_{1}\cr z_{2}\cr z_{3}\cr ...\cr y_{n}\end{bmatrix} 
 = {\begin{bmatrix}1&x_{1}&x_{1}^{2}&... &x_{1}^{r}\cr1&x_{2}&x_{2}^{2}&... &x_{2}^{r}\cr1&x_{3}&x_{3}^{2}&... &x_{3}^{r}\cr...\cr1&x_{n}&x_{n}^{2}&\dots &x_{n}^{r}
                  \cr......................................&...............................&................................&.............
                  \cr 1 &y_{1}&y_{1}^{2}&... &y_{1}^{r}\cr 1 &y_2 &y_{2}^{2}&... &y_{2}^{r}\cr 1 &y_{3}&y_{3}^{2}&... &y_{3}^{r}\cr...\cr 1&y_{n}&y_{n}^{2}&\dots &y_{n}^{r}\end{bmatrix}}
 {\begin{bmatrix}\beta _{00}&\beta _{01}&\beta _{02}&\vdots &\beta _{0r}
                  \cr &&...& &\beta_{1r}
                  \cr\beta_{r0}&... &...& &\beta_{rr}\end{bmatrix}}$


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#THIS FUNCTIONS GIVES AN ARRAY OF A COMPONENT, GIVING EACH DEGREE POWERED BY COLUMNS

#variables definition
def power_populate(init:float,end:float,N_samp:int, deg=int):
    Vec = np.array(np.linspace(init,end,N_samp)).reshape(-1,1)
    Vec_poly = np.hstack((np.ones(Vec.shape),Vec))  
    # return Vec
    for i in range(2,deg+1):
        a = np.power(i, Vec)#.reshape(Vec.shape[0],1) #, Vec
        Vec_poly = np.hstack((Vec_poly,a))
    return Vec,Vec_poly

X1,X1_poly = power_populate(-1,1,100, deg=2)
X2,X2_poly = power_populate(-1,1,100, deg=2)


#expanding to 3d array:
#   1st  (hor):  degree of polynomial
#   2nd  (ver):  number of measurement
#   3th  (deepness): each independant variable
X_3d = np.array([X1_poly,X2_poly])
X1_2d,X2_2d = np.meshgrid(X1,X2)

In [ ]:
### CHECKPOINT!!
### CHECKPOINT!!
### CHECKPOINT!!
### CHECKPOINT!!
### CHECKPOINT!!


A = np.array([[0,1,-1],   #a00,...a0g
              [0,-3,0],   
              [-1,1,3]]) #ag0,...agg 

#matriz Y representa valores escalares no plano/¿ou hiperplano de N dim?
def polyn_calcul(coeff_mat,x,y,):     #queremos calcular aqui 
    grade = coeff_mat.shape[0]
    x_powers = [np.array(x**i) for i in range(grade)]# for var in (x,y)]
    y_powers= np.array([y**i for i in range(grade)])# for var in (x,y)]
    return x_powers,y_powers
    return  x + x**2 - 3*x*y -y**2 + y**2*x +3*x**2*y**2

x,y = polyn_calcul(A, X1_2d,X2_2d)
# Y_2d = polyn_calcul(X1_2d,X2_2d)


In [ ]:
#plotting cell
__,ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X1_2d, X2_2d, Y_2d)

<br><br> Putting the independent and dependent variables in matrices ${\displaystyle X}$ and ${\displaystyle Y}$, respectively, we can compute the least squares in the following way. Note that ${\displaystyle D}$ is the set of all data.

${\displaystyle L(D,{\boldsymbol {\beta }})=\left\|Y-X{\boldsymbol {\beta }}\right\|^{2}=(Y-X{\boldsymbol {\beta }})^{\mathsf {T}}(Y-X{\boldsymbol {\beta }})=Y^{\mathsf {T}}Y-Y^{\mathsf {T}}X{\boldsymbol {\beta }}-{\boldsymbol {\beta }}^{\mathsf {T}}X^{\mathsf {T}}Y+{\boldsymbol {\beta }}^{\mathsf {T}}X^{\mathsf {T}}X{\boldsymbol {\beta }}}$

The gradient of the loss is:
${\displaystyle {\frac {\partial L(D,{\boldsymbol {\beta }})}{\partial {\boldsymbol {\beta }}}}={\frac {\partial \left(Y^{\mathsf {T}}Y-Y^{\mathsf {T}}X{\boldsymbol {\beta }}-{\boldsymbol {\beta }}^{\mathsf {T}}X^{\mathsf {T}}Y+{\boldsymbol {\beta }}^{\mathsf {T}}X^{\mathsf {T}}X{\boldsymbol {\beta }}\right)}{\partial {\boldsymbol {\beta }}}}=-2X^{\mathsf {T}}Y+2X^{\mathsf {T}}X{\boldsymbol {\beta }}}$

Setting the gradient of the loss to zero and solving for β {\displaystyle {\boldsymbol {\beta }}}, we get:<br>
${\displaystyle -2X^{\mathsf {T}}Y+2X^{\mathsf {T}}X{\boldsymbol {\beta }}=0\Rightarrow X^{\mathsf {T}}Y=X^{\mathsf {T}}X{\boldsymbol {\beta }}}$<br><br>
${\displaystyle {\boldsymbol {\hat {\beta }}}=\left(X^{\mathsf {T}}X\right)^{-1}X^{\mathsf {T}}Y}$

In [ ]:
# Col1, Col2 = power_populate(-1,1,100, deg=2)


betha = np.matmul(np.linalg.inv(np.matmul(X.transpose(), X)) * X.transpose() , Yre)



END OF LATE VERSION


THERE IS A DIMENSIONAL GAP BETWEEN BOTH PARTS: <br><BR>
UPSIDE WE DEFINED EACH SAMPLE AS A VECTOR, AND EXPAND THE EQUATIONS AS:  **THE POWER OF THE VECTOR TO THE EXPONENT OF EACH DEGREE**<BR>
BELOW WE DEFINING EACH SAMPLE AS THE TUPLE OF VECTOR COMPONENTS, AND EXPAND THE EQUATIONS AS:  **THE POWER OF EACH COMPONENT OF THE VECTOR TO THE EXPONENT OF EACH DEGREE**

### Polynomial Surface:
Lets express the equation of a polynomial surfaces in 2 ways: as double summatory and in a matricial form. <br><br>
$\quad \quad{\displaystyle \sum _j^g \sum _i^g a_{ij} x^i y^j} \quad = \quad X*A*Y$ 
<br>
${X = \begin{bmatrix}x^0&...&x^i&...&x^g \end{bmatrix}}$
$\quad$
${A = \begin{bmatrix}a_{00}&a_{01}&...&a_{0g}\cr a_{10}&...&&a_{1g}  \cr ...\cr a_{g0}&...&&a_{gg} \end{bmatrix}}$
$\quad$
${Y = \begin{bmatrix}y^0\cr ...\cr y^j\cr ...\cr y^g \end{bmatrix}}$



In [ ]:
#variables definition
X = np.linspace(-1,1,1000); Y = np.linspace(-1,1,1000)  #vector of 100 samples of each independete variable

A = np.array([[0,1,-1],   #a00,...a0g
              [0,-3,0],   
              [-1,1,3]]) #ag0,...agg 


Z = np.zeros((len(X),len(Y)))

for n,x in enumerate(X):
    for m,y in enumerate(Y):
        x_poly=np.matrix([x**i for i in range(A.shape[0])])
        y_poly = np.matrix([y**i for i in range( A.shape[0])]).reshape(-1,1)
        
        #here we perfomr the matrix operation {X*A*Y} for each sample of X,Y measurements 
        aux =np.matmul(x_poly,A,)
        Z[n,m] = np.matmul( aux,y_poly)
             


In [ ]:
#plotting cell
__,ax = plt.subplots(subplot_kw={"projection": "3d"})
X,Y = np.meshgrid(X,Y)
surf = ax.plot_surface(X, Y, Z,)